In [1]:
import numpy as np
import pandas as pd
import redis
import cv2

In [2]:
r = redis.StrictRedis(
    host="redis-11755.c308.sa-east-1-1.ec2.cloud.redislabs.com", 
    port=11755,
    username="default", # use your Redis user. More info https://redis.io/docs/management/security/acl/
    password="CzdAM8ReaYrKSySrvShim7abK4ZENraC", # use your Redis password
)

In [3]:
r.ping()

True

In [4]:
# load the data (npzip)
np_file = np.load('dataframe_students_teachers.npz', allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

df = pd.DataFrame(x_values, columns=col_names)
df

,Name,Role,Facial_Features
0,Angelina Jolie,student,"[0.43242034, 1.0748498, -0.12862368, -1.833291..."
1,Barack Obama,teacher,"[0.94858104, -0.5584052, 0.094820715, -1.27965..."
2,Chris Evans,student,"[0.7201716, -1.4684149, -0.7781254, -1.4601072..."
3,Morgan Freeman,teacher,"[0.485351, -0.6192554, -0.4069932, 1.0687786, ..."
4,Scarlett Johansson,student,"[0.19131963, -0.48843804, -1.690233, 0.4423967..."


In [5]:
df['name_role'] = df['Name']+'@'+df['Role']
df

,Name,Role,Facial_Features,name_role
0,Angelina Jolie,student,"[0.43242034, 1.0748498, -0.12862368, -1.833291...",Angelina Jolie@student
1,Barack Obama,teacher,"[0.94858104, -0.5584052, 0.094820715, -1.27965...",Barack Obama@teacher
2,Chris Evans,student,"[0.7201716, -1.4684149, -0.7781254, -1.4601072...",Chris Evans@student
3,Morgan Freeman,teacher,"[0.485351, -0.6192554, -0.4069932, 1.0687786, ...",Morgan Freeman@teacher
4,Scarlett Johansson,student,"[0.19131963, -0.48843804, -1.690233, 0.4423967...",Scarlett Johansson@student


In [6]:
records = df[['name_role', 'Facial_Features']].to_dict(orient='records')
len(records)

5

### Save data in redis
* redis hashes
* keyname = 'academy:register'

In [7]:
for record in records:
    name_role = record['name_role']
    vector = record['Facial_Features']

    # convert np array into bytes
    vector_bytes = vector.tobytes()
    r.hset('academy:register', key=name_role, value=vector_bytes)

# Registration form

In [8]:
import cv2
from insightface.app import FaceAnalysis

In [9]:
#config face analysis
faceapp = FaceAnalysis(name='buffalo_l',
                       root='models',
                       providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])

faceapp.prepare(ctx_id=0,
                det_size=(640, 640),
                det_thresh=0.5)

/home/glccampos/Downloads/pass/envs/face_reco/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


### 1st step collect person role and name

In [10]:
# py input
person_name = input('Enter name: ')
trials = 3
for i in range(trials):
    role = input("""
    Please choose:
                1. Student
                2. Teacher

                Enter either 1 or 2
                """)
    if role in ('1', '2'):
        if role == '1':
            role = 'Student'
        else:
            role = 'Teacher'
        break
    else:
        print('Invalid entry')
        if i == 3:
            print('Exceed max attempts!')

key = person_name + '@' + role
print('Your name =', person_name)
print('Your role', role)
print('key =', key)

Your name = Lopes
Your role Student
key = Lopes@Student


In [11]:
person_name, role

('Lopes', 'Student')

### 2nd step collect samples

In [16]:
cap = cv2.VideoCapture(0)
face_embeddings = []
samples = 0

while True:
    ret, frame = cap.read()
    if ret == False:
        print('Unable to read camera')
        break
    # get results from insightface model
    results = faceapp.get(frame, max_num=1)
    for res in results:
        samples += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 1)

        #facial embeddings
        embeddings = res['embedding']
        face_embeddings.append(embeddings)
    
    if samples >= 200:
        break

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

/home/glccampos/Downloads/pass/envs/face_reco/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


### 3rd step optimize data and save in redis

In [19]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [20]:
# convert into bytes
x_mean_bytes = x_mean.tobytes()

In [21]:
# save key and facial bytes into redis db
r.hset(name='academy:register', key=key, value=x_mean_bytes)

1